In [8]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import scipy
from scipy.stats import norm

# Independent Sample t-Test and A/B Test

## Theory

**Central Limit Theory**
- ${Z_1, Z_2,...}$ from $iid$
- $E(Z) = \mu, Var(Z) = \sigma^2$
- When sample size is big:
$$\frac{\bar Z -\mu}{\sigma / \sqrt N} \xrightarrow{N\to\infty} N(0,1)$$
$$\frac{\bar Z -\mu}{\hat \sigma / \sqrt N} \xrightarrow{N\to\infty} N(0,1)$$

- When $Z$ comes from normal distribution:
$$\frac{\bar Z -\mu}{\sigma / \sqrt N} \sim N(0,1)$$
$$\frac{\bar Z -\mu}{\hat \sigma / \sqrt N} \sim t(N-1)$$

**Assumptions**:
1. X and Y are independent
1. X and Y have same variance $\sigma^2$
1. X and Y from normal distribution, respectively
---

**Binomial Distribution**
       
- For single observation
    - $\sigma^2 = p(1-p)$
    - $\hat \sigma^2 = s^2 = \hat p (1-\hat p)$
    
    
- For average/ratio/proportion:
    - $\hat p = \bar X = (x_1 + x_2 + ... + x_n) / N $
    - $Var(\hat p) = Var(\bar X) = \frac{p(1-p)}{N}$
    - $s^2(\hat p) = s^2(\bar X) =\frac{\hat p (1-\hat p)}{N}$


- For difference in average/ratio/proportion (same sample size):
    - $\bar X$ and $\bar Y$: $Var(\bar X - \bar Y) = \frac{2p(1-p)}{N} $
    - $\bar X$ and $\bar Y$: $s^2(\bar X - \bar Y) = \frac{2\hat p(1-\hat p)}{N} $
    
    

**Test Statistic**
$$t = \frac{(\bar X - \bar Y) - (\mu_X - \mu_Y)}{\hat \sigma _{\bar X - \bar Y}}$$

$$\hat \sigma _{\bar X - \bar Y} = {S_{\bar X - \bar Y}} = s_p\sqrt{1/N_X+1/N_Y} \xrightarrow{N_X = N_Y}  s_p \sqrt \frac{2}{N}$$


$$s_p^2 = \frac{(N_x-1)\hat \sigma_X^2 + (N_y-1)\hat \sigma_y^2}{N_X+N_Y-2}\xrightarrow{N_X = N_Y} \frac{1}{2}(\sigma_X^2 + \sigma_Y^2)\xrightarrow{\sigma_X^2 = \sigma_Y^2} \sigma^2$$

- Under Normal Distribution (***Independent Sample T-test***): 
    $$\hat \sigma_X = S_X,\hat \sigma_Y = S_Y $$
    
    
- Under Binomial Distribution (***A/B Test***): $$\hat \sigma_X = \hat \sigma_Y = \sqrt{\hat p(1-\hat p)}$$ 


- $\ t- distribution\ (df=m+n-2)$

**Violation of assumptions**

- For $1^{st}$ assumption: by experiment design

- For $2^{nd}$ assumption: Perform *Levene test *
    * Null hypothesis: samples have same variances
    * Reject null hypothesis when $p<\alpha=0.05$
    * When violated, the calculation of $df$ will change

- For 3rd assumption: Perform *Shapiro-Wilks test*
    - Reject null hypothesis when $p<\alpha=0.05$
    - When sample size is big, still valid (asymptotic normality)
    - Reason: Central Limit Theory for $\bar X$ and $\bar Y$

**Relationship with Likelihood-Ratio Test**
- Can be proved to be equivalent

# Numerical Example


## Import Data

Data source: https://github.com/nirupamaprv/Analyze-AB-test-Results

In [9]:
df = pd.read_csv('./data/ab_edited.csv')[:50]
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [10]:
old = df.query("group == 'control'")['converted']
new = df.query("group == 'treatment'")['converted']

In [11]:
p_old = np.mean(old)
p_old

0.23809523809523808

In [12]:
# example for independent sample t-test
#old=[79.98,80.04,80.02,80.04,80.03,80.03,80.04,79.97,80.05,80.03,80.02,80.00,80.02]
#new=[80.02,79.94,79.98,79.97,79.97,80.03,79.95,79.97]

## Implementation with `stats`

### Assuming normal distribution, t-test

In [13]:
t_score, p_value = scipy.stats.ttest_ind(old, new, equal_var=True)
t_score, p_value 

(0.25771350650932573, 0.7977297197070108)

### Assume binomial distribution, A/B test

In [14]:
convert_old = sum(old)
convert_new = sum(new)
n_old = len(df.query("group == 'control'"))
n_new = len(df.query("group == 'treatment'"))

In [15]:
z_score, p_value = sm.stats.proportions_ztest([convert_old, convert_new], 
                                              [n_old, n_new], 
                                              prop_var = p_old)
z_score, p_value

(0.2556432050257977, 0.7982263538405426)

## Implementation with `numpy` only

- Perform *Shapiro-Wilks* Normality Test

In [16]:
from scipy.stats import shapiro
shapiro(old),shapiro(new)

((0.5327093601226807, 4.1332060618515243e-07),
 (0.4997923970222473, 7.758476527897074e-09))

- **p-value**: less than significance level: 0.05
- **Conclusion**: The normnal assumption is violated 

- Perform *Levene* Test

In [17]:
from scipy.stats import levene
levene(old, new)

LeveneResult(statistic=0.06641625143733229, pvalue=0.7977297197070145)

- **p-value**: larger than significance level: 0.05
- **Conclusion**: X and Y have same population variance

- Calculate $\bar{X}$ and $\bar{Y}$

In [18]:
mean_old = np.mean(old) 
mean_new = np.mean(new) 
mean_old, mean_new

(0.23809523809523808, 0.20689655172413793)

- Under Normal: Calculate $S_x$ and $S_y$

In [19]:
s_old = np.std(old, ddof=1) 
s_new = np.std(new, ddof=1) 
s_old, s_new

(0.4364357804719847, 0.4122508203948855)

- Under Binomial: Calculate $\hat \sigma_X$  and $\hat \sigma_Y$

In [20]:
s_old = s_new = np.sqrt(p_old * (1 - p_old))
s_old, s_new

(0.4259177099999599, 0.4259177099999599)

- Calculate $N_x$ and $N_y$

In [21]:
n_old = len(old) 
n_new = len(new) 
n_old, n_new

(21, 29)

- Calculate DoF
    - With same variance, $df = N_x + N_y - 2$
    - Note: more complicated calculations if $\sigma^2_1 \neq \sigma^2_2$

In [22]:
df1 = n_old - 1
df2 = n_new - 1
df = df1 + df2
df

48

- Calculate pooled variance

In [23]:
s_p = np.sqrt(( (n_old - 1) * s_old * s_old + (n_new - 1) * s_new * s_new) / (n_old + n_new - 2))
s_p

0.4259177099999599

- Calculate test statistics

In [24]:
T = (mean_old - mean_new) /(s_p * np.sqrt(1 / n_old + 1 / n_new))
T

0.2556432050257977

- Calculate p-value (Single-tail)

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/41/Student_t_pdf.svg/325px-Student_t_pdf.svg.png" width="400">

<img src="http://www.ttable.org/uploads/2/1/7/9/21795380/published/9754276.png?1517416376" width="400">

- Use t distribution

In [25]:
from scipy.stats import t
t.cdf(T, df=df)

0.6003406522596397

In [30]:
p = 1 - (t.cdf(T, df=df)-0.5) * 2
p

0.7993186954807205

- Use normal distribution

In [147]:
from scipy.stats import norm
print(norm.cdf(T))

0.6008868230797286


In [148]:
p = 1 - (norm.cdf(T)-0.5) * 2
p

0.7982263538405427

Conclusion: **CANNOT** reject $H_0$ that old and new has same conversion ratio

# Calculate sample size / Power

Power: 
- $P$(reject $H_0$|$H_1$ is true)
- Red shaded area


<img src="https://i.ytimg.com/vi/70uNTAP1J-I/maxresdefault.jpg" width="700">

Example: https://www.evanmiller.org/ab-testing/sample-size.html

In [1]:
p_baseline = 0.2 # under H_0
effect_size = 0.05 # Desired effect size
sig = 0.95 # assume two-tail
sample_size = 1030

- look up table: $Z(\alpha/2) = 1.96$


- Calculate power of test
    - Standardize user-provided $ES$ 
    $$ ES_{N(0,1)}  = \frac{ES}{\hat \sigma_{\bar X - \bar Y}} = \frac{ES}{\hat \sigma}\sqrt{\frac{N}{2}}$$
$$$$
    - Calculate the arrow point:
    $$ - ES_{N(0,1)} + Z_{critical}$$
    $$$$
    
    - Calculate the area of light-red
    $$\phi_z(- ES_{N(0,1)} + Z_{critical})$$
    $$$$
    - Calculate the area of dark red
    $$1-\phi_z(- ES_{N(0,1)} + Z_{critical})$$
    
    

In [80]:
s_x = np.sqrt(p_baseline * (1 - p_baseline))
s_x

0.4

In [81]:
s_diff =  s_x * np.sqrt( 2 / sample_size)
s_diff

0.01762610596956927

In [82]:
effect_size_N_0_1 = effect_size / s_diff
effect_size_N_0_1

2.836701429477554

In [83]:
phi_value = 1.96 - effect_size_N_0_1
phi_value

-0.8767014294775541

In [84]:
red_shade = norm.cdf(phi_value)
red_shade

0.19032441506917608

In [85]:
power = 1 - red_shade
power

0.8096755849308239

# Case Study

## Problem Statement
- Given a feature difference in facebook app, evaluate if the change will improve user activity.
- Given a UI component change (e.g., button color) in a pageview, evaluate if there are more users clicking.
- Given a pop-up message, whether users will continue enroll in program or not
- Given a new firewall feature in GCP

http://rajivgrover1984.blogspot.com/2015/11/ab-testing-overview.html
>*For example: An online education company tested a change where if the student clicked "start free trial", they were asked how much time they had available to devote to the course. If the student indicated 5 or more hours per week, they would be taken through the checkout process as usual. If they indicated fewer than 5 hours per week, a message would appear indicating that these courses usually require a greater time commitment for successful completion, and suggesting that the student might like to access the course materials for free.*

## Choose Subject (Unit of diversion)


Possible Choice: 
- User id
- Cookie
- Event

## Choose metric

Example of pop-up message and program enrollment

Metrics should **NOT** change:



- Number of cookies, unique # of cookies visiting the page
- Number of clicks on the button (since message shown after clicking)

Metrics that **MAY** change:


- $p = \frac{Number\ of\ users\ actually\ enrolled}{Number\ of\ users\ clicking\ button}$


- $p = \frac{Number\ of\ users\ remain\ enrolled\ for\ 14\ days}{Number\ of\ users\ clicking\ button}$

## Choose size and duration

see above

## Perform sanity check and result analysis

See above

# Network effect

- Sample consistency, for example, GCP, two uses in one collaboration group faces two different features. Or adding a video chatting feature, which only works if both sides have access to it
- Sample independency (Spillover effect), for example, Facebook: many connected components, thus Group A and B are no longer independent.

- Possible solution: community (cluster) based AB test by partitioning nodes to groups, or for a dating app with no prior connections, maybe using demographic/geographical attributes
- Each **cluster** is assigned a treatment, thus unlikely for spillover from control to treatment
- Unit of analysis is reduced, higher varinace as a result

Ref: http://web.media.mit.edu/~msaveski/projects/2016_network-ab-testing.html

<img src="http://web.media.mit.edu/~msaveski/assets/projects/2016_network-ab/main.png" width="500">
